In [1]:
import sys
import os

# Add project root to the system path
project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))
if project_root not in sys.path:
    sys.path.append(project_root)


# Loading experiment parameters

In [2]:
from src.utils.yaml_loader import read_yaml

settings = read_yaml('../../experiments/gridloss/feedforward.yml')
settings

{'dataset_args': {'directory': '/Users/vegardsjavik/Uni/Master/Thesis/BEAST/data/gridloss',
  'training_data': 'train.csv',
  'test_data': 'test.csv',
  'window_size': 24,
  'timeseries_to_use': ['grid1-loss', 'grid1-load', 'grid1-temp']},
 'model_args': {'window_size': 24,
  'network_size': [100, 100],
  'horizon_length': 24,
  'model_name': 'feedforward',
  'model_params_storage_dir': '/Users/vegardsjavik/Uni/Master/Thesis/BEAST/models/feedforward'},
 'training_args': {'batch_size': 16, 'epochs': 1000, 'learning_rate': 0.001}}

In [3]:
learning_rate = settings['training_args']['learning_rate']
window_size = settings['model_args']['window_size']
horizon_length = settings['model_args']['horizon_length']
features_to_use = settings['dataset_args']['timeseries_to_use']
model_save_dir = settings['model_args']['model_params_storage_dir']

input_size = window_size*len(features_to_use)
output_size = horizon_length

# Creating model

In [4]:
from src.models.feedforward import NN

model = NN(
    input_size=input_size,
    output_size=output_size,
    learning_rate=learning_rate,
    save_dir=model_save_dir
)

Loading trained model from /Users/vegardsjavik/Uni/Master/Thesis/BEAST/models/feedforward/feedforward_forecaster.pth...


/Users/vegardsjavik/Uni/Master/Thesis/BEAST/src/models/feedforward.py:84: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = load(self.save_dir)
